```
bryan@ichabod mac_bucket % for file in *_core.json; do
  databricks fs cp "$file" dbfs:/Volumes/geodata/petra/core/
done
```

In [ ]:
df = spark.read.format('json').load("/Volumes/geodata/petra/core/")

display(df)

In [ ]:
from common.transforms import string_to_iso_date, generate_hash
import pyspark.sql.functions as F

df_flat = df.select(
    F.col("repo_id"),
    F.col("uwi.uwi").alias("uwi"),
    F.col("uwi.wsn").alias("wsn"),
    F.col("cores.base").alias("base"),
    F.col("cores.top").alias("top"),
    F.col("cores.date").alias("date"),
    F.col("cores.desc").alias("desc"),
    F.col("cores.fmname").alias("formation"),
    F.col("cores.qual").alias("qualifier"),
    F.col("cores.recover").alias("recover"),
    F.col("cores.type").alias("type")
)

date_columns = [
    "date"
]

df_core = df_flat


# enforce timestamp for dates
for col_name in date_columns:
    df_core = string_to_iso_date(df_core, col_name, col_name)

# add id hash
id_columns = ["repo_id", "uwi"]
df_core = generate_hash(df_core, "id", "core", *id_columns)


display(df_core)